<a href="https://colab.research.google.com/github/csalnav2/QdotCS/blob/master/Unsloth_Solutions_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Unsloth (Demo Solutions Notebook)

This notebook collects various code snippets that address specific tasks:

1. **nF4 → Triton** (Quantized 4-bit kernel demo)
2. **QLoRA + `torch.compile`** (Naive QLoRA example, no graph breaks)
3. **QLoRA + FSDP** (Fully Sharded Data Parallel + LoRA injection)
4. **Memory-Efficient Backprop** (Chunked final linear + cross-entropy)
5. **Windows Support** (Python scripts to build/install `unsloth`, plus test code)
6. **Flexible Attention** ("Unsloth" style chunked attention examples)
7. **Sequence Classification Patch** (Inject LoRA into `AutoModelForSequenceClassification`)
8. **Refactored Attention** (xformers, SDPA, flash-attn, fallback in one interface)

Feel free to skip cells or modify as needed.

---
## 1) **nF4 → Triton**

**Goal**: Demonstrate converting 4-bit weights (nF4 style) and using a Triton kernel to do matrix multiplication without fully decompressing everything into float16/float32 first.

**Note**: This code is a **minimal skeleton**. Real nF4 implementations might have more complex scaling logic, per-row or per-channel quant parameters, etc.

In [1]:
# If in Google Colab or a fresh environment, install specific versions:
# (You can comment these out if you already have matching versions.)
!pip uninstall -y torch triton
!pip install --no-cache-dir torch==2.1.0+cu121 torchvision==0.16.0+cu121 torchaudio==2.1.0+cu121 --index-url https://download.pytorch.org/whl/cu121
!pip install --no-cache-dir triton==2.1.0

import torch
import triton
import triton.language as tl

print("PyTorch version:", torch.__version__)
print("Triton version:", triton.__version__)

# ---------------------------------------------------------------------
# Hard-coded tile sizes => compile-time constants
# ---------------------------------------------------------------------
BLOCK_M = 64
BLOCK_N = 64
BLOCK_K = 64

@triton.jit
def nf4_tile_matmul(
    A_ptr, B_ptr, C_ptr,
    M, N, K,
    stride_am, stride_ak,
    stride_bk, stride_bn,
    stride_cm, stride_cn,
    BLOCK_M: tl.constexpr,
    BLOCK_N: tl.constexpr,
    BLOCK_K: tl.constexpr
):
    """
    Single-tile NF4 decode + partial matmul.
    - M,N,K => must each be 64 exactly.
    - A => shape [M, K//2] in 4-bit. B => shape [K, N//2].
    - Output => [M, N] float16 in C.

    We'll decode each nibble from A, B => cast to float32 => do dot => store float16.
    """
    # Decode A => shape [BLOCK_M, BLOCK_K]
    rowA = tl.arange(0, BLOCK_M)
    colA = tl.arange(0, BLOCK_K)
    rowA = rowA[:, None]  # shape [BLOCK_M,1]
    colA = colA[None, :]  # shape [1,BLOCK_K]
    linearA = rowA*stride_am + colA*stride_ak
    byteA   = linearA // 2
    nibSelA = linearA & 1
    bytesA  = tl.load(A_ptr + byteA)
    shiftA  = nibSelA * 4
    nibA    = (bytesA >> shiftA) & 0xF
    valA    = tl.cast(nibA, tl.float32)

    # Decode B => shape [BLOCK_K, BLOCK_N]
    rowB = tl.arange(0, BLOCK_K)
    colB = tl.arange(0, BLOCK_N)
    rowB = rowB[:, None]  # shape [BLOCK_K,1]
    colB = colB[None, :]  # shape [1,BLOCK_N]
    linearB = rowB*stride_bk + colB*stride_bn
    byteB   = linearB // 2
    nibSelB = linearB & 1
    bytesB  = tl.load(B_ptr + byteB)
    shiftB  = nibSelB * 4
    nibB    = (bytesB >> shiftB) & 0xF
    valB    = tl.cast(nibB, tl.float32)

    # partial dot => shape [BLOCK_M, BLOCK_N]
    accum = tl.dot(valA, valB)

    # store to C => shape [BLOCK_M, BLOCK_N]
    out_f16 = accum.to(tl.float16)
    rowC = tl.arange(0, BLOCK_M)[:, None]
    colC = tl.arange(0, BLOCK_N)[None, :]
    c_offset = rowC*stride_cm + colC*stride_cn
    tl.store(C_ptr + c_offset, out_f16)

def nf4_tile_matmul_host(A_4bit, B_4bit, M, N, K):
    """
    Host function: A_4bit => [M, K//2], B_4bit => [K, N//2], each nibble = 4 bits
    Output => [M, N] float16.

    M,N,K must be 64 to match the tile kernel.
    """
    device = A_4bit.device
    C = torch.empty((M, N), dtype=torch.float16, device=device)

    # row-major strides
    stride_am = K
    stride_ak = 1
    stride_bk = N
    stride_bn = 1
    stride_cm = N
    stride_cn = 1

    # Single tile => (1,1) grid
    nf4_tile_matmul[(1,1)](
        A_4bit, B_4bit, C,
        M, N, K,
        stride_am, stride_ak,
        stride_bk, stride_bn,
        stride_cm, stride_cn,
        BLOCK_M=BLOCK_M,
        BLOCK_N=BLOCK_N,
        BLOCK_K=BLOCK_K
    )
    return C

# ---------------------------------------------------------------------
# Demo usage
# ---------------------------------------------------------------------
device = "cuda"
M, K, N = 64, 64, 64  # must match BLOCK_M,N,K=64

A_4bit = torch.randint(0, 256, (M, K//2), dtype=torch.uint8, device=device)
B_4bit = torch.randint(0, 256, (K, N//2), dtype=torch.uint8, device=device)

C_out = nf4_tile_matmul_host(A_4bit, B_4bit, M, N, K)
print("C_out shape:", C_out.shape)
print("C_out[:5, :5] =>\n", C_out[:5, :5])
print("Done. If 'map::at' error appears, it's likely a Triton environment bug.")


Found existing installation: torch 2.1.0+cu121
Uninstalling torch-2.1.0+cu121:
  Successfully uninstalled torch-2.1.0+cu121
Found existing installation: triton 2.1.0
Uninstalling triton-2.1.0:
  Successfully uninstalled triton-2.1.0
Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 GB 271.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 319.1 MB/s eta 0:00:00
PyTorch version: 2.1.0+cu121
Triton version: 2.1.0


AssertionError: libcuda.so cannot found!


For the following code regarding the nf4 to Triton conversion, this was the output on A100  on Google Colab but Google colab for some strange reason cannot recognize libcuda.so. The code stopped working and did not save the output. But, here is the actual out put for the code above. This code should work in differennt Colab, I think there is a glitch in Google colab. Currently trying to reproduce the ecexutation of this code in a different Colan environment.
PyTorch version: 2.5.1+cu124
Triton version: 3.1.0
C_out shape: torch.Size([64, 64])
C_out[:5, :5] =>
 tensor([[3144., 3652., 3420., 2944., 3216.],
        [3230., 3672., 3480., 3682., 3348.],
        [2744., 3316., 3248., 3372., 2804.],
        [3776., 3744., 3800., 3614., 3416.],
        [3976., 4264., 3874., 3886., 3506.]], device='cuda:0',
       dtype=torch.float16)

In [ ]:
# Single cell for Colab or Jupyter
# 1) Force reinstall pinned Torch/Triton from PyTorch's cu121 index
# 2) If Transformers 4.32+ not found on PyPI for Py 3.11 => installs from source (latest master).
# 3) Attempt bitsandbytes unpinned (remove if wheels are missing).

print("=== [1] Uninstall old Torch/Transformers/bitsandbytes/peft/accelerate/trl ===")
!pip uninstall -y torch triton transformers bitsandbytes peft accelerate trl

print("\n=== [2] Install Torch 2.1.0+cu121 & Triton 2.1.0 from PyTorch's cu121 index ===")
INSTALL_CMD_TORCH = """
pip install --upgrade --force-reinstall --no-cache-dir \
  torch==2.1.0+cu121 torchvision==0.16.0+cu121 torchaudio==2.1.0+cu121 \
    --index-url https://download.pytorch.org/whl/cu121 \
  triton==2.1.0
"""
print(INSTALL_CMD_TORCH)
res = get_ipython().system(INSTALL_CMD_TORCH)

print("\n=== [3] Attempting Transformers >=4.32 from PyPI, bitsandbytes unpinned, plus peft/accelerate/trl pinned ===")
INSTALL_CMD_PYPI = """
pip install --upgrade --force-reinstall --no-cache-dir \
  "transformers>=4.32.0,<5.0.0" \
  bitsandbytes \
  peft==0.5.0 \
  accelerate==0.23.0 \
  trl==0.6.0
"""
print(INSTALL_CMD_PYPI)
res = get_ipython().system(INSTALL_CMD_PYPI)

if res != 0:
    print("\nPyPI install for Transformers >=4.32 or bitsandbytes might have failed. Attempting source install for Transformers.\n")
    # 4) If Transformers fails from PyPI, we do a source install from Git
    #    This ensures we have the latest code with is_torch_less_than_1_11.
    #    But if bitsandbytes still fails, you may need to remove it or build from source.
    INSTALL_CMD_SRC = """
pip install --force-reinstall --no-cache-dir git+https://github.com/huggingface/transformers.git@main peft==0.5.0 accelerate==0.23.0 trl==0.6.0 bitsandbytes
"""
    print(INSTALL_CMD_SRC)
    get_ipython().system(INSTALL_CMD_SRC)

print("\n=== [4] Show final installed versions ===")
!pip show torch
!pip show transformers
!pip show bitsandbytes
!pip show peft
!pip show trl
!pip show accelerate

print("\n=== [5] Attempt `import transformers.trainer` ===")
try:
    import transformers
    print(f"transformers version => {transformers.__version__}")
    from transformers import trainer
    print("SUCCESS: `transformers.trainer` imported => 'is_torch_less_than_1_11' error is gone!")
except ImportError as e:
    print("ImportError =>", e)
except Exception as ex:
    print("Unexpected error =>", ex)


=== [1] Uninstall old Torch/Transformers/bitsandbytes/peft/accelerate/trl ===
Found existing installation: torch 2.1.0+cu121
Uninstalling torch-2.1.0+cu121:
  Successfully uninstalled torch-2.1.0+cu121
Found existing installation: triton 3.2.0
Uninstalling triton-3.2.0:
  Successfully uninstalled triton-3.2.0

=== [2] Install Torch 2.1.0+cu121 & Triton 2.1.0 from PyTorch's cu121 index ===

pip install --upgrade --force-reinstall --no-cache-dir   torch==2.1.0+cu121 torchvision==0.16.0+cu121 torchaudio==2.1.0+cu121     --index-url https://download.pytorch.org/whl/cu121   triton==2.1.0

Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 GB 295.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 211.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 279.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 299.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━


=== [3] Attempting Transformers >=4.32 from PyPI, bitsandbytes unpinned, plus peft/accelerate/trl pinned ===

pip install --upgrade --force-reinstall --no-cache-dir   "transformers>=4.32.0,<5.0.0"   bitsandbytes   peft==0.5.0   accelerate==0.23.0   trl==0.6.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 113.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 238.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 245.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 154.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 260.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 128.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 268.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 152.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.0/110.0 kB 308.8 MB/s eta 0:00:00
  


PyPI install for Transformers >=4.32 or bitsandbytes might have failed. Attempting source install for Transformers.


pip install --force-reinstall --no-cache-dir git+https://github.com/huggingface/transformers.git@main peft==0.5.0 accelerate==0.23.0 trl==0.6.0 bitsandbytes

  Cloning https://github.com/huggingface/transformers.git (to revision main) to /tmp/pip-req-build-8x7auish
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-8x7auish
  Resolved https://github.com/huggingface/transformers.git to commit e18f233f6c8cba029324e2868fb68abdaf6badf3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 248.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 247.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 118.7 MB/s eta 0


=== [4] Show final installed versions ===
Name: torch
Version: 2.6.0
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3-Clause
Location: /usr/local/lib/python3.11/dist-packages
Requires: filelock, fsspec, jinja2, networkx, nvidia-cublas-cu12, nvidia-cuda-cupti-cu12, nvidia-cuda-nvrtc-cu12, nvidia-cuda-runtime-cu12, nvidia-cudnn-cu12, nvidia-cufft-cu12, nvidia-curand-cu12, nvidia-cusolver-cu12, nvidia-cusparse-cu12, nvidia-cusparselt-cu12, nvidia-nccl-cu12, nvidia-nvjitlink-cu12, nvidia-nvtx-cu12, sympy, triton, typing-extensions
Required-by: accelerate, bitsandbytes, fastai, peft, sentence-transformers, timm, torchaudio, torchvision, trl
Name: transformers
Version: 4.50.0.dev0
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future)

/usr/local/lib/python3.11/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.11/dist-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


SUCCESS: `transformers.trainer` imported => 'is_torch_less_than_1_11' error is gone!


---
## 2) **QLoRA + `torch.compile`** (Naive Example)

This snippet demonstrates a simple QLoRA-like module (4-bit quant + LoRA adapters), then wraps the model in `torch.compile` to ensure we avoid graph breaks.

In [2]:

#!/usr/bin/env python3

"""
Full Modified Code - QLoRA + Torch.compile for MLP, RMSNorm, Attention, and final Loss
--------------------------------------------------------------------------------------
- Installs bitsandbytes if missing
- Loads a LLaMA 7B model in 4-bit
- Applies LoRA
- Monkey-patches LlamaAttention, LlamaMLP, LlamaRMSNorm, and final forward+loss
  to be compiled with torch.compile
- Simple data sample & training loop
"""

import sys
import subprocess
import os

##############################################################################
# 0) Optionally install bitsandbytes if missing
##############################################################################
def ensure_bitsandbytes_installed():
    try:
        import bitsandbytes
        print("bitsandbytes already installed:", bitsandbytes.__version__)
    except ModuleNotFoundError:
        print("\n[!] bitsandbytes not found. Installing now...\n")
        cmd = [
            sys.executable, "-m", "pip", "install",
            "--upgrade", "--force-reinstall", "bitsandbytes"
        ]
        subprocess.run(cmd, check=False)

ensure_bitsandbytes_installed()

##############################################################################
# 1) Some environment and logging configs
##############################################################################
os.environ["TORCHDYNAMO_VERBOSE"] = "1"
os.environ["TORCHINDUCTOR_FORCE_DISABLE_CACHES"] = "1"
os.environ["TORCHINDUCTOR_COMPILE_THREADS"] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
import logging
torch._inductor.config.debug = True
torch._logging.set_logs(
    dynamo=logging.WARN,
    inductor=logging.WARN,
    graph_breaks=True,
    recompiles=True,
)

torch._dynamo.config.verbose = True
torch._dynamo.config.suppress_errors = False

print("PyTorch version:", torch.__version__)

import transformers
print("Transformers version:", transformers.__version__)

##############################################################################
# 2) Import bitsandbytes & the rest
##############################################################################
import bitsandbytes as bnb
print("bitsandbytes version:", bnb.__version__)

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)
from peft import get_peft_model, LoraConfig, TaskType

##############################################################################
# 3) Load a LLaMA model in 4-bit (QLoRA style)
##############################################################################
model_name = "decapoda-research/llama-7b-hf"  # or any LLaMA
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config,
    attn_implementation="flash",  # or "sdpa"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)
if not tokenizer.pad_token:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

##############################################################################
# 4) Apply LoRA
##############################################################################
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
    target_modules=[
        "q_proj","k_proj","v_proj","o_proj",
        "gate_proj","up_proj","down_proj",
    ],
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

model.enable_input_require_grads()
model.train()

##############################################################################
# 5) Monkey-patch Attention, RMSNorm, MLP, and final forward+loss
#    so that they're compiled with torch.compile
##############################################################################
import transformers.models.llama.modeling_llama as llama_mod

# A) RMSNorm
@torch.compile(fullgraph=False, dynamic=True,
    options={"max_autotune":True, "shape_padding":True}
)
def compiled_llama_rmsnorm_forward(self, hidden_states):
    variance = hidden_states.float().pow(2).mean(-1, keepdim=True)
    hidden_states = hidden_states * torch.rsqrt(variance + self.eps)
    return (hidden_states.to(self.weight.dtype) * self.weight)

llama_mod.LlamaRMSNorm.forward = compiled_llama_rmsnorm_forward

# B) Llama MLP
@torch.compile(fullgraph=False, dynamic=True,
    options={"max_autotune":True, "shape_padding":True}
)
def compiled_llama_mlp_forward(self, x):
    return self.down_proj(self.act_fn(self.gate_proj(x)) * self.up_proj(x))

llama_mod.LlamaMLP.forward = compiled_llama_mlp_forward

# C) Llama Attention (simplified)
@torch.compile(fullgraph=False, dynamic=True,
    options={"max_autotune":True, "shape_padding":True}
)
def compiled_llama_attention_forward(self, hidden_states, attention_mask=None, position_ids=None):
    bsz, q_len, _ = hidden_states.size()
    query_states = self.q_proj(hidden_states)
    key_states   = self.k_proj(hidden_states)
    value_states = self.v_proj(hidden_states)

    # Basic matmul attention (not real flash code, but for illustration)
    attn_weights = torch.matmul(query_states, key_states.transpose(-1,-2))
    if attention_mask is not None:
        attn_weights = attn_weights + attention_mask
    attn_weights = torch.softmax(attn_weights, dim=-1)

    attn_output = torch.matmul(attn_weights, value_states)
    output = self.o_proj(attn_output)
    return output

llama_mod.LlamaAttention.forward = compiled_llama_attention_forward

# D) final forward + loss
@torch.compile(fullgraph=False, dynamic=True,
    options={"max_autotune":True, "shape_padding":True}
)
def compiled_forward_with_loss(model, input_ids, labels):
    outputs = model(input_ids, labels=labels)
    return outputs.loss

def train_step_compiled(model, optimizer, input_ids, labels):
    loss = compiled_forward_with_loss(model, input_ids, labels)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    return loss.item()

##############################################################################
# 6) Toy training loop
##############################################################################
from datasets import load_dataset
url = "https://huggingface.co/datasets/laion/OIG/resolve/main/unified_chip2.jsonl"
print("\n=== Loading a small dataset sample ===")
ds = load_dataset("json", data_files={"train": url}, split="train[:0.01%]")  # 0.01% for speed
print("Dataset length:", len(ds))

from torch.optim import AdamW
optimizer = AdamW((p for p in model.parameters() if p.requires_grad), lr=1e-4)

max_steps = 10
step = 0

for sample in ds:
    if step >= max_steps:
        break
    text = sample["text"] if "text" in sample else "Hello world"
    enc = tokenizer(
        text,
        padding="max_length",
        max_length=64,
        truncation=True
    )
    input_ids = torch.tensor(enc["input_ids"], device=model.device).unsqueeze(0)
    labels = input_ids.clone()

    loss_val = train_step_compiled(model, optimizer, input_ids, labels)
    step += 1
    print(f"Step {step}, Loss = {loss_val:.4f}")

print("\n=== Done! ===")
print("Compiled Attention, RMSNorm, MLP, and final forward+loss. bitsandbytes calls will still produce graph breaks.")




[!] bitsandbytes not found. Installing now...



ImportError: cannot import name 'justknobs_check' from 'torch._utils_internal' (/usr/local/lib/python3.11/dist-packages/torch/_utils_internal.py)

---
## 3) **QLoRA + FSDP**

A single-cell script that:
- Loads BERT in half precision
- Injects LoRA modules
- Wraps the model in FSDP (Fully Sharded Data Parallel)
- Trains only the LoRA parameters

In [3]:
#!/usr/bin/env python3

"""
Max-Points Example for QLoRA + Inductor:
---------------------------------------
1. Flexible (flash/sdpa) attention => dynamic sequence lengths OK.
2. Partial/no compile for bitsandbytes => we skip the bnb kernel calls
   that rely on data_ptr().
3. We do compile:
   - Llama attention
   - Llama MLP
   - Llama RMSNorm (layernorm)
   - The final loss
4. We use 'max_autotune' and 'dynamic=True' in torch.compile options
   to allow shape changes without repeated recompiles.
5. We attempt to avoid "excessive_recompilation" by controlling
   batch shapes & sequence lengths carefully in the training loop.

Caveats:
- bitsandbytes 4-bit calls will still produce graph breaks
  because they do .data_ptr().
- So we only do partial compile around them. We'll monkey-patch
  the Llama attention, MLP, RMSNorm, and the final forward pass
  that includes the loss.
"""

import sys
import subprocess
import os
import logging

##############################################################################
# 0) (Optional) Cleanup old packages
##############################################################################
def optional_cleanup():
    # If you prefer not to uninstall anything automatically, skip this.
    old_pkgs = ["torch", "transformers", "peft", "bitsandbytes", "accelerate", "trl"]
    cmd = [sys.executable, "-m", "pip", "uninstall", "-y"] + old_pkgs
    print("\n=== [A] Uninstalling old pkgs (optional) ===")
    subprocess.run(cmd)

##############################################################################
# 1) Install or check versions
##############################################################################
def install_required():
    # Pinned or minimum versions for best results
    needed = [
        "torch>=2.1.0",
        "transformers>=4.33.0",  # for flash/sdpa & Llama2
        "peft>=0.5.0",
        "bitsandbytes>=0.45.0",
        "accelerate>=0.23.0",
        "trl>=0.6.0",
        "datasets>=2.14.0",
    ]
    cmd = [sys.executable, "-m", "pip", "install", "--upgrade"] + needed
    print("\n=== [B] Installing required pkgs ===")
    print("CMD:", cmd)
    subprocess.run(cmd, check=False)

if __name__ == "__main__":
    # optional_cleanup()
    # install_required()
    pass

##############################################################################
# 2) Env + logging
##############################################################################
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"
os.environ["TORCHDYNAMO_VERBOSE"] = "1"
os.environ["TORCHINDUCTOR_FORCE_DISABLE_CACHES"] = "1"
os.environ["TORCHINDUCTOR_COMPILE_THREADS"] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # or "0,1"

import torch

torch._inductor.config.debug = True
torch._logging.set_logs(
    dynamo=      logging.WARN,
    inductor=    logging.WARN,
    graph_breaks=True,
    recompiles=  True,
)

torch._dynamo.config.verbose = True
torch._dynamo.config.suppress_errors = False

print("PyTorch version:", torch.__version__)

##############################################################################
# 3) Library imports
##############################################################################
import transformers
print("Transformers version:", transformers.__version__)
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

from peft import get_peft_model, LoraConfig, TaskType
import bitsandbytes as bnb
print("bitsandbytes version:", bnb.__version__)

##############################################################################
# 4) Load 4-bit model with "flex"/"flash"/"sdpa" attention
##############################################################################
model_name = "decapoda-research/llama-7b-hf"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

print(f"\n=== Loading LLaMA model {model_name} in 4-bit ===")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config,
    attn_implementation="flash",  # or "sdpa" or "native"
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
    target_modules=[
        "q_proj","k_proj","v_proj","o_proj",
        "gate_proj","up_proj","down_proj",
    ],
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# By default, only LoRA layers need grad
model.enable_input_require_grads()

##############################################################################
# 5) We monkey-patch the attention, RMSNorm, MLP, and final loss
#    so that they're compiled. We'll skip bitsandbytes calls.
##############################################################################
import transformers.models.llama.modeling_llama as llama_mod

# A) RMSNorm (LlamaRMSNorm)
@torch.compile(fullgraph=False, dynamic=True,
    options={"max_autotune":True, "shape_padding":True, "trace.enabled":True}
)
def compiled_llama_rmsnorm_forward(self, hidden_states):
    """
    Original HF code:
      variance = hidden_states.to(torch.float32).pow(2).mean(-1, keepdim=True)
      hidden_states = hidden_states * torch.rsqrt(variance + self.eps)
      return hidden_states.to(self.weight.dtype) * self.weight
    We'll keep the same logic. We'll rely on float32 for numeric stability.
    """
    variance = hidden_states.float().pow(2).mean(-1, keepdim=True)
    hidden_states = hidden_states * torch.rsqrt(variance + self.eps)
    return (hidden_states.to(self.weight.dtype) * self.weight)

llama_mod.LlamaRMSNorm.forward = compiled_llama_rmsnorm_forward

# B) Llama MLP (like your previous example)
@torch.compile(fullgraph=False, dynamic=True,
    options={"max_autotune":True, "shape_padding":True}
)
def compiled_llama_mlp_forward(self, x):
    # Original:
    # out = self.down_proj( self.act_fn(self.gate_proj(x)) * self.up_proj(x) )
    return self.down_proj( self.act_fn(self.gate_proj(x)) * self.up_proj(x) )

llama_mod.LlamaMLP.forward = compiled_llama_mlp_forward

# C) Llama Attention patch. We'll do a partial patch that compiles the
#    math in the attention forward, skipping the bitsandbytes step if any.
#    If we do "attn_implementation='flash'", HF uses a flash-attn-like path
#    we can try to compile as well.
#    We'll just patch the entire "LlamaAttention.forward" for demonstration.
@torch.compile(fullgraph=False, dynamic=True,
    options={"max_autotune":True, "shape_padding":True}
)
def compiled_llama_attention_forward(self, hidden_states, attention_mask=None, position_ids=None):
    """
    This is truncated. In reality, Llama uses a flash-attn or sdp path if
    attn_implementation='flash'. We'll skip bitsandbytes calls if any occur.
    """
    # Original code logic:
    bsz, q_len, _ = hidden_states.size()
    # Project hidden_states to QKV
    query_states = self.q_proj(hidden_states)
    key_states   = self.k_proj(hidden_states)
    value_states = self.v_proj(hidden_states)
    # Possibly call flash-attn or F.scaled_dot_product_attention...
    # We'll do a minimal example to illustrate partial compile:
    # shape => (bsz, q_len, num_heads, head_dim)
    # We'll skip real-sdp detail for brevity:
    # We'll do a simple matmul-based attention (not truly flash).
    #
    # [bsz, q_len, num_heads * head_dim]
    # just a naive version:
    attn_weights = torch.matmul(query_states, key_states.transpose(-1,-2))
    # apply mask
    if attention_mask is not None:
        attn_weights = attn_weights + attention_mask
    attn_weights = torch.softmax(attn_weights, dim=-1)
    # apply
    attn_output = torch.matmul(attn_weights, value_states)
    # output proj:
    output = self.o_proj(attn_output)
    return output

llama_mod.LlamaAttention.forward = compiled_llama_attention_forward

# D) Compile the final forward + loss.
#    We'll define a custom method that does the forward pass + crossent,
#    then we compile that.
#    In training, we can call *only this compiled* function to ensure
#    the loss is included in the graph.
@torch.compile(fullgraph=False, dynamic=True,
    options={"max_autotune":True, "shape_padding":True, "trace.enabled":True}
)
def compiled_forward_with_loss(model, input_ids, labels):
    # We do a standard forward pass => ignoring bitsandbytes calls inside.
    # Then compute cross-entropy in the same graph:
    outputs = model(input_ids, labels=labels)
    loss = outputs.loss  # cross_entropy
    return loss

# We define a small helper to do the step:
def train_step_compiled(model, optimizer, input_ids, labels):
    # forward
    loss = compiled_forward_with_loss(model, input_ids, labels)
    # backward
    loss.backward()
    # step
    optimizer.step()
    optimizer.zero_grad()
    return loss.item()

##############################################################################
# 6) A small training loop on a toy dataset => watch for recompile
##############################################################################
from datasets import load_dataset
dataset_url = "https://huggingface.co/datasets/laion/OIG/resolve/main/unified_chip2.jsonl"
print("\n=== [6] Loading a small dataset sample ===")
ds = load_dataset("json", data_files={"train": dataset_url}, split="train[:0.01%]")
print("Dataset length:", len(ds))

from torch.optim import AdamW
optimizer = AdamW([p for p in model.parameters() if p.requires_grad], lr=1e-4)

# We'll do a simple example training loop:
torch.manual_seed(42)
model.train()

# Our naive small-batch loop:
max_steps = 10
step = 0
for sample in ds:
    if step >= max_steps:
        break
    # tokenization (short, no special prompts, for demonstration)
    enc = tokenizer(sample["text"], padding="max_length", max_length=64, truncation=True)
    input_ids = torch.tensor(enc["input_ids"], device=model.device).unsqueeze(0)
    labels = input_ids.clone()

    # This calls our compiled forward with cross-entropy
    loss_val = train_step_compiled(model, optimizer, input_ids, labels)
    step += 1
    print(f"Step {step}, Loss = {loss_val:.4f}")

print("\n=== Done! ===")
print("We compiled attention, MLP, RMSNorm, and the final forward+loss.")
print("We used flash attn to handle dynamic sequence lengths. We used partial compile to skip bitsandbytes calls.")
print("Check logs for potential minimal recompile or graph breaks.")



PyTorch version: 2.5.1+cu124
Transformers version: 4.48.3


ModuleNotFoundError: No module named 'bitsandbytes'

---
## 4) **Memory-Efficient Backprop** (Chunked Final MatMul + Cross-Entropy)

This code chunk demonstrates how to avoid creating a huge `[B*S, vocab]` logits matrix at once, by chunking the matmul into smaller pieces. This reduces memory usage at the cost of multiple partial computations.

In [2]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

##############################################################################
# 1) The Transformation Function
#    - Projects a chunk X_chunk => logits, then cross-entropy w.r.t label_chunk
##############################################################################
def transformation_function(X_chunk, linear_module, label_chunk):
    """
    1) X_chunk shape: [chunk_size, hidden_dim]
    2) linear_module: an nn.Linear (hidden_dim -> vocab_size)
    3) label_chunk:   [chunk_size]
    Returns a scalar cross-entropy loss (mean).
    """
    # For numerical stability, cast logits to float32
    # in case X_chunk or linear_module are half precision
    logits = linear_module(X_chunk).float()
    ce_fn = nn.CrossEntropyLoss(reduction="mean")
    loss = ce_fn(logits, label_chunk)
    return loss

##############################################################################
# 2) Custom Autograd Function for Memory-Efficient Final Linear
##############################################################################
class MemoryEfficientLinear(torch.autograd.Function):
    @staticmethod
    def forward(ctx, X, linear_module, labels, forward_fn, chunk_size=1024):
        """
        X:            [batch_size * seq_len, hidden_dim]
        linear_module: an nn.Linear or similar final projection
        labels:       [batch_size * seq_len]
        forward_fn:   function(X_chunk, linear_module, label_chunk) => scalar
        chunk_size:   how many rows to process per chunk
        """
        device = X.device
        # We'll accumulate the total loss as a scalar
        total_loss = torch.zeros([], device=device, dtype=X.dtype)

        num_rows = X.shape[0]
        # For each chunk, we compute partial loss, then accumulate
        for start in range(0, num_rows, chunk_size):
            end = min(start + chunk_size, num_rows)
            X_chunk = X[start:end]
            lbl_chunk = labels[start:end]

            loss_chunk = forward_fn(X_chunk, linear_module, lbl_chunk)
            # Weighted average if chunk sizes differ
            frac = (end - start) / float(num_rows)
            total_loss = total_loss + loss_chunk * frac

        # Save necessary info for backward
        ctx.save_for_backward(X, labels)
        ctx.linear_module = linear_module
        ctx.forward_fn = forward_fn
        ctx.chunk_size = chunk_size

        return total_loss

    @staticmethod
    def backward(ctx, grad_output):
        """
        We re-run the chunked forward pass to compute dX, dWeights, etc.
        grad_output is shape [], the derivative wrt. the final scalar.
        We'll scale each chunk's loss by grad_output.
        """
        (X, labels) = ctx.saved_tensors
        linear_module = ctx.linear_module
        forward_fn = ctx.forward_fn
        chunk_size = ctx.chunk_size

        grad_multiplier = grad_output.item()  # scalar for entire loss

        # We'll accumulate dX in a buffer
        dX = torch.zeros_like(X)

        # Zero the linear_module's param grads
        # because we'll do partial chunk .backward() calls
        linear_module.zero_grad()

        num_rows = X.shape[0]
        for start in range(0, num_rows, chunk_size):
            end = min(start + chunk_size, num_rows)
            # Make a sub-tensor that we can differentiate
            X_chunk = X[start:end].detach().requires_grad_(True)
            lbl_chunk = labels[start:end]

            loss_chunk = forward_fn(X_chunk, linear_module, lbl_chunk)
            frac = (end - start) / float(num_rows)
            scaled_loss = loss_chunk * frac * grad_multiplier
            scaled_loss.backward()

            dX[start:end] = X_chunk.grad

        # We return grad for X, linear_module, labels, forward_fn, chunk_size
        #  - X => dX
        #  - linear_module => None (it's not a Tensor, param grads are in linear_module's .grad)
        #  - labels => None (no grad needed)
        #  - forward_fn => None
        #  - chunk_size => None
        return dX, None, None, None, None

##############################################################################
# 3) Demo Usage
##############################################################################
def demo_memory_efficient_linear():
    """
    We'll do a large (batch_size=4, seq_len=4096, hidden_dim=4096, vocab=128k)
    but for demonstration, we'll do smaller shapes.
    Adjust shapes as needed.
    """
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print("Running on:", device)

    # Example shapes (tweak them to something big if you want to test memory)
    batch_size = 4
    seq_len = 4096
    hidden_dim = 4096
    vocab_size = 128000
    chunk_size = 1024  # adjust for VRAM needs

    # We'll create random X, labels
    X = torch.randn(batch_size*seq_len, hidden_dim, device=device, dtype=torch.float16, requires_grad=True)
    labels = torch.randint(0, vocab_size, (batch_size*seq_len,), device=device)

    # Build a linear projection
    linear_module = nn.Linear(hidden_dim, vocab_size).to(device, dtype=torch.float16)

    # Forward pass via MemoryEfficientLinear
    loss = MemoryEfficientLinear.apply(X, linear_module, labels, transformation_function, chunk_size)
    print("Loss:", loss.item())

    # Backward => chunk-based approach
    loss.backward()
    print("X grad shape:", X.grad.shape)
    print("linear_module.weight grad shape:", linear_module.weight.grad.shape)

    # You can check VRAM usage or time as well.

if __name__ == "__main__":
    demo_memory_efficient_linear()


Running on: cuda
Loss: 11.927106857299805


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [ ]:
# ============================================
# 1) Confirm GPU type (T4, A100, etc.).
# ============================================
!nvidia-smi

# ============================================
# 2) [Optional] Install system-level CUDA 11.8 libs
#    so bitsandbytes can find libcusparse.so.11, etc.
#    If you get 'libcusparse.so.11 not found' errors,
#    installing these packages often helps.
# ============================================
!apt-get update -y
!apt-get install -y --no-install-recommends \
    cuda-cudart-11-8 \
    cuda-cusparse-11-8 \
    cuda-libraries-11-8

# ============================================
# 3) Wipe older Torch/bitsandbytes/xformers/triton
#    to avoid conflicts.
# ============================================
!pip uninstall -y torch bitsandbytes xformers triton

# ============================================
# 4) Install PyTorch 2.0.1+cu118, matching torchvision/torchaudio.
# ============================================
!pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2+cu118 \
    --extra-index-url https://download.pytorch.org/whl/cu118

# ============================================
# 5) (Optional) Re-install pinned bitsandbytes, xformers, triton
#    to confirm environment is consistent.
#    (Though build_unsloth.py may also install them depending on the markers.)
# ============================================
!pip install bitsandbytes==0.41.1 xformers==0.0.22 triton==2.0.0 \
    --extra-index-url https://download.pytorch.org/whl/cu118


Thu Feb 20 16:26:18 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   30C    P0             45W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

---
## 5) **Windows Support**

Below are two scripts:
- **`build_unsloth.py`**: Creates a `pyproject.toml`, builds a wheel, and installs it.
- **`test_deps.py`**: Installs bitsandbytes, xformers, triton, then tests them.

These are primarily relevant for letting `unsloth` (and associated libraries) build on Windows.

In [ ]:
%%writefile build_unsloth.py
import os
import sys
import subprocess

# 1) Write pyproject.toml with correct license syntax, allowing Python 3.9+
toml_content = """\
[project]
name = "unsloth"
version = "0.1.0"
description = "unsloth: Windows-friendly package for bitsandbytes, xformers, triton"
readme = "README.md"
requires-python = ">=3.9"

[project.license]
text = "MIT"

authors = [
  { name = "Your Name", email = "you@example.com" }
]

# Dependencies only install if environment markers match (e.g., Windows).
# On Colab Linux + CUDA 11.8, these might not do anything,
# but we still define them to show the "Windows-friendly" idea.
dependencies = [
  "torch==2.0.1+cu118; platform_system=='Windows'",
  "transformers==4.30.2",
  "accelerate==0.20.3",
  "bitsandbytes==0.39.1",
  "xformers==0.0.20",
  "triton==2.0.0",
]

[build-system]
requires = ["setuptools>=61", "wheel"]
build-backend = "setuptools.build_meta"
"""

with open("pyproject.toml", "w", encoding="utf-8") as f:
    f.write(toml_content)

# 2) Minimal package structure
os.makedirs("src/unsloth", exist_ok=True)
with open("src/unsloth/__init__.py", "w", encoding="utf-8") as f:
    f.write('# unsloth package init - minimal\n')

# Minimal README
with open("README.md", "w", encoding="utf-8") as f:
    f.write("# unsloth\n\nA Windows-friendly package with bitsandbytes, xformers, triton.\n")

print("=== pyproject.toml created. Attempting to build and install locally... ===")

# 3) Upgrade pip and install build tools
subprocess.run([
    "python", "-m", "pip", "install", "--upgrade",
    "pip", "build", "setuptools>=61", "wheel"
], check=True)

# 4) Build the wheel
build_result = subprocess.run(["python", "-m", "build"], capture_output=True, text=True)
if build_result.returncode != 0:
    print("ERROR: Build failed. Output:\n")
    print(build_result.stdout)
    print(build_result.stderr)
    sys.exit(1)

# 5) Check dist/ directory
if not os.path.isdir("dist"):
    print("ERROR: 'dist/' directory not found, build likely failed.")
    sys.exit(1)

dist_files = os.listdir("dist")
if not dist_files:
    print("ERROR: 'dist/' directory is empty, no wheel found.")
    sys.exit(1)

wheel_files = [f for f in dist_files if f.endswith(".whl")]
if not wheel_files:
    print("ERROR: No .whl file found in dist/. Found:", dist_files)
    sys.exit(1)

wheel_path = os.path.join("dist", wheel_files[0])

# 6) Install the wheel with extra index for cu118
cmd = [
    "python",
    "-m",
    "pip",
    "install",
    wheel_path,
    "--extra-index-url",
    "https://download.pytorch.org/whl/cu118"
]
print("\nInstalling wheel with command:", " ".join(cmd))

install_result = subprocess.run(cmd, capture_output=True, text=True)
if install_result.returncode != 0:
    print("ERROR: Failed to install the wheel. Output:\n")
    print(install_result.stdout)
    print(install_result.stderr)
    sys.exit(1)

print("Successfully installed the unsloth wheel from dist/!\n")
print("Installation log:")
print(install_result.stdout)

# ============== End of build_unsloth.py ==============


Overwriting build_unsloth.py


In [ ]:
!python build_unsloth.py

=== pyproject.toml created. Attempting to build and install locally... ===
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 64.3 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.1.0
    Uninstalling setuptools-75.1.0:
      Successfully uninstalled setuptools-75.1.0
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.

Installing wheel with command: python -m pip install dist/unsloth-0.1.0-py3-none-any.whl --extra-index-url https://download.pytorch.org/whl/cu118

Installation log:
Looking in indexes: https://pypi.org/simple, https://dow

In [ ]:
################################################################################
# ONE-CELL COLAB SCRIPT: PyTorch Nightly (2.2.0 + cu121),
# bitsandbytes 0.45.2, xformers 0.0.24, tested on A100 CUDA 12.x
################################################################################

print("=== Checking GPU and driver info ===")
!nvidia-smi

print("\n=== 1) Uninstall older Torch, bitsandbytes, xformers, triton ===")
!pip uninstall -y torch bitsandbytes xformers triton

print("\n=== 2) Install PyTorch NIGHTLY 2.2.0+cu121, plus torchvision, torchaudio")
print("         from the official 'nightly/cu121' index. ===")

# We use --pre (pre-release) and a special index URL for nightly cu121 builds.
!pip install --pre torch torchvision torchaudio \
    --index-url https://download.pytorch.org/whl/nightly/cu121

print("\n=== 3) Install bitsandbytes 0.45.2 and xformers 0.0.24 (built for Torch 2.2.0+cu121) ===")
# We'll just use PyPI. bitsandbytes 0.45.2 has CUDA 12.1 support.
# xformers 0.0.24 is built for Torch 2.2.0+cu121, so it won't conflict.
!pip install bitsandbytes==0.45.2 xformers==0.0.24

print("\n=== 4) Write test_deps.py script to verify bitsandbytes, xformers, and triton ===")

test_deps_code = """import os
import sys
import torch

os.environ["BNB_CUDA_VERSION"] = "121"  # bitsandbytes tries libbitsandbytes_cuda121.so

# 1) Test bitsandbytes
try:
    import bitsandbytes as bnb
    print("\\n=== bitsandbytes import OK ===")
    linear_8bit = bnb.nn.Linear8bitLt(128, 64).cuda()
    dummy_in = torch.randn(16, 128, device='cuda', dtype=torch.float16)
    dummy_out = linear_8bit(dummy_in)
    print('bitsandbytes linear8bit forward pass successful. Output shape:', dummy_out.shape)
except Exception as ex:
    print('bitsandbytes usage error:', ex)
    sys.exit(1)

# 2) Test xformers
try:
    import xformers
    print("\\n=== xformers import OK ===")
    from xformers.ops import fmha
    q = torch.randn((1, 32, 8, 64), device='cuda', dtype=torch.float16)
    k = torch.randn((1, 32, 8, 64), device='cuda', dtype=torch.float16)
    v = torch.randn((1, 32, 8, 64), device='cuda', dtype=torch.float16)
    out = fmha.memory_efficient_attention(q, k, v)
    print('xformers fmha output shape:', out.shape)
except Exception as ex:
    print('xformers usage error:', ex)
    sys.exit(1)

# 3) Test triton (bundled in Torch 2.2.0 nightly)
try:
    import triton
    import triton.language as tl
    print("\\n=== triton import OK ===")

    @triton.jit
    def add_kernel(x_ptr, y_ptr, output_ptr, BLOCK_SIZE: tl.constexpr):
        pid = tl.program_id(0)
        offset = pid * BLOCK_SIZE + tl.arange(0, BLOCK_SIZE)
        x = tl.load(x_ptr + offset)
        y = tl.load(y_ptr + offset)
        tl.store(output_ptr + offset, x + y)

    x_t = torch.randn(1024, device='cuda')
    y_t = torch.randn(1024, device='cuda')
    output_t = torch.empty(1024, device='cuda')
    grid = (1024 // 256,)
    add_kernel[grid](x_t, y_t, output_t, BLOCK_SIZE=256)
    print('triton add_kernel test, first 5 results:', output_t[:5].tolist())
except Exception as ex:
    print('triton usage error:', ex)
    sys.exit(1)

print('\\nAll tests passed! bitsandbytes, xformers, and triton are working.')
"""

with open("test_deps.py", "w") as f:
    f.write(test_deps_code)

print("\n=== 5) Run test_deps.py to confirm everything works with Torch 2.2.0+cu121 ===")
!python test_deps.py


=== Checking GPU and driver info ===
Thu Feb 20 17:19:27 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   30C    P0             45W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+----------

---
## 6) **Flexible Attention**

Here’s a snippet that builds various attention masks (causal, sliding, etc.) and uses a chunked approach, plus `torch.compile` if you like. This demonstration shows different mask types in one place.

In [ ]:
import sys
import math
import torch

def build_attention_mask(seq_len, mask_type="causal", window_size=64, device="cuda"):
    """
    Creates an attention mask:
      - "causal": blocks j > i (standard auto-regressive mask).
      - "sliding": local window = ±window_size around each token.
    """
    mask = torch.zeros(seq_len, seq_len, device=device)
    if mask_type == "causal":
        # Triangular upper matrix => block j>i
        casual_mat = torch.triu(torch.ones(seq_len, seq_len, device=device), diagonal=1)
        mask[casual_mat.bool()] = float("-1e9")
    elif mask_type == "sliding":
        # For each position i, block everything outside [i - window_size, i + window_size]
        for i in range(seq_len):
            left = max(0, i - window_size)
            right = min(seq_len, i + window_size + 1)
            mask[i, :left] = float("-1e9")
            mask[i, right:] = float("-1e9")
    else:
        raise ValueError(f"Unknown mask_type={mask_type}")
    return mask

def flex_attention(q, k, v, attn_mask):
    """
    Simple scaled dot-product attention:
      q, k, v: shape [batch, seq_len, d_model]
      attn_mask: shape [seq_len, seq_len], large negative => blocked
    """
    d_model = q.shape[-1]
    # (batch, seq_len, d_model) @ (batch, d_model, seq_len) => (batch, seq_len, seq_len)
    attn_scores = torch.bmm(q, k.transpose(1, 2)) / math.sqrt(d_model)

    # Apply the mask (broadcast => (batch, seq_len, seq_len))
    attn_scores = attn_scores + attn_mask.unsqueeze(0)

    # Softmax and multiply by v
    attn_probs = torch.softmax(attn_scores, dim=-1)
    out = torch.bmm(attn_probs, v)
    return out

# Fallback approach for Python 3.11:
# - If Python < 3.11 => we compile
# - If Python >= 3.11 => skip compile to avoid runtime error
if sys.version_info < (3, 11):
    compiled_flex_attention = torch.compile(flex_attention, mode="default")
    print("Using torch.compile on Python < 3.11.")
else:
    compiled_flex_attention = flex_attention
    print("Skipping torch.compile (Python 3.11+ not yet supported).")

def run_flex_attention_demo():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    batch_size = 2
    d_model = 64

    for mask_type in ["causal", "sliding"]:
        print(f"\n===> Testing mask_type = {mask_type}")
        for seq_len in [128, 256, 300, 512]:
            q = torch.randn(batch_size, seq_len, d_model, device=device)
            k = torch.randn(batch_size, seq_len, d_model, device=device)
            v = torch.randn(batch_size, seq_len, d_model, device=device)

            base_mask = build_attention_mask(seq_len, mask_type=mask_type, device=device)
            out = compiled_flex_attention(q, k, v, base_mask)
            print(f"seq_len={seq_len}, out.shape={out.shape}, mask_type={mask_type}")

if __name__ == "__main__":
    run_flex_attention_demo()


Skipping torch.compile (Python 3.11+ not yet supported).

===> Testing mask_type = causal
seq_len=128, out.shape=torch.Size([2, 128, 64]), mask_type=causal
seq_len=256, out.shape=torch.Size([2, 256, 64]), mask_type=causal
seq_len=300, out.shape=torch.Size([2, 300, 64]), mask_type=causal
seq_len=512, out.shape=torch.Size([2, 512, 64]), mask_type=causal

===> Testing mask_type = sliding
seq_len=128, out.shape=torch.Size([2, 128, 64]), mask_type=sliding
seq_len=256, out.shape=torch.Size([2, 256, 64]), mask_type=sliding
seq_len=300, out.shape=torch.Size([2, 300, 64]), mask_type=sliding
seq_len=512, out.shape=torch.Size([2, 512, 64]), mask_type=sliding


---
## 7) **Sequence Classification Patch** (LoRA + `AutoModelForSequenceClassification`)

We patch `AutoModelForSequenceClassification` by injecting LoRA modules into every `nn.Linear` in the model, then fine-tune only the LoRA parameters on a toy dataset.

In [ ]:
################################################################################
# SINGLE-CELL COLAB SCRIPT:
# LoRA BERT classification w/ Torch 2.1.0+cu121 & Transformers 4.31.0
# Removing peft & older libraries => fix the 'adapter_kwargs' error.
################################################################################

print("=== Checking GPU / driver info ===")
!nvidia-smi

print("\n=== 1) Uninstall conflicting packages (torch, transformers, peft, xformers, etc.) ===")
!pip uninstall -y torch transformers peft xformers tokenizers bitsandbytes

print("\n=== 2) Install Torch 2.1.0+cu121 & Transformers==4.31.0 ===")
!pip install torch==2.1.0+cu121 --index-url https://download.pytorch.org/whl/cu121
!pip install transformers==4.31.0

print("\n=== 3) Running your LoRA BERT classification code ===")

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    AutoConfig,
)

class ToyClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=32):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt"
        )
        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "labels": torch.tensor(label, dtype=torch.long)
        }

class LoRALinear(nn.Module):
    def __init__(self, in_features, out_features, lora_rank=4, alpha=1.0):
        super().__init__()
        self.lora_down = nn.Linear(in_features, lora_rank, bias=False)
        self.lora_up   = nn.Linear(lora_rank, out_features, bias=False)
        nn.init.zeros_(self.lora_down.weight)
        nn.init.zeros_(self.lora_up.weight)
        self.alpha = alpha

    def forward(self, x):
        return self.alpha * self.lora_up(self.lora_down(x))

def patch_model_for_sequence_classification(model, lora_rank=4, alpha=1.0):
    modules_to_patch = []
    for name, module in model.named_modules():
        if isinstance(module, nn.Linear):
            modules_to_patch.append((name, module))

    for full_name, module in modules_to_patch:
        safe_name = full_name.replace(".", "_")
        lora_mod = LoRALinear(
            module.in_features,
            module.out_features,
            lora_rank=lora_rank,
            alpha=alpha
        ).to(module.weight.device, module.weight.dtype)

        # Register it
        model.add_module(f"lora_{safe_name}", lora_mod)

        # Patch forward
        orig_forward = module.forward
        def custom_forward(m_self, x, orig_forward=orig_forward, lora_layer=lora_mod):
            base_out = orig_forward(x)
            lora_out = lora_layer(x)
            return base_out + lora_out

        module.forward = custom_forward.__get__(module, module.__class__)

    return model

def finetune_sequence_classification():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model_name = "bert-base-uncased"
    num_labels = 2

    config = AutoConfig.from_pretrained(model_name, num_labels=num_labels)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, config=config)
    model.to(device)

    # Inject LoRA
    patch_model_for_sequence_classification(model, lora_rank=4, alpha=1.0)

    texts = [
        "I love this product, it is amazing!",
        "This is the worst experience of my life.",
        "The movie was quite entertaining.",
        "Horrible service, will not come back!"
    ]
    labels = [1, 0, 1, 0]
    dataset = ToyClassificationDataset(texts, labels, tokenizer, max_length=16)
    dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

    # Only train LoRA params
    lora_params = []
    for param_name, param in model.named_parameters():
        if "lora_" in param_name:
            param.requires_grad = True
            lora_params.append(param)
        else:
            param.requires_grad = False

    optimizer = optim.AdamW(lora_params, lr=1e-4)
    model.train()
    epochs = 3
    for epoch in range(epochs):
        total_loss = 0.0
        for batch in dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            optimizer.zero_grad()
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss / len(dataloader)
        print(f"Epoch {epoch+1}/{epochs}, avg_loss={avg_loss:.4f}")

    model.eval()
    sample_text = ["I dislike the taste, not recommended."]
    enc = tokenizer(sample_text, truncation=True, padding=True, return_tensors="pt").to(device)
    with torch.no_grad():
        logits = model(**enc).logits
    preds = torch.argmax(logits, dim=-1)
    print("\nInference Test:")
    print(f"Input: {sample_text}")
    print(f"Logits: {logits.cpu().numpy()}")
    print(f"Predicted label: {preds.item()} (0=Neg,1=Pos)")

finetune_sequence_classification()


=== Checking GPU / driver info ===
Thu Feb 20 18:43:30 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   30C    P0             45W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+------------

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probab

Epoch 1/3, avg_loss=0.8157
Epoch 2/3, avg_loss=0.7862
Epoch 3/3, avg_loss=0.7097

Inference Test:
Input: ['I dislike the taste, not recommended.']
Logits: [[-0.36591572  0.18960014]]
Predicted label: 1 (0=Neg,1=Pos)


---
## 8) **Refactored Attention**

Merging `xformers`, PyTorch’s SDPA, `flash_attn`, and a fallback “flex” approach in a single function.

In [ ]:
import warnings

try:
    import xformers.ops as xops
    XFORMERS_AVAILABLE = True
except ImportError:
    XFORMERS_AVAILABLE = False

try:
    import flash_attn
    FLASH_ATTN_AVAILABLE = True
except ImportError:
    FLASH_ATTN_AVAILABLE = False

SDPA_AVAILABLE = hasattr(torch.nn.functional, "scaled_dot_product_attention")

def flex_custom_attention(q, k, v, attn_mask=None):
    d_k = q.shape[-1]
    scores = torch.matmul(q, k.transpose(-1, -2)) / (d_k ** 0.5)
    if attn_mask is not None:
        scores = scores + attn_mask
    weights = torch.softmax(scores, dim=-1)
    weights = weights.to(v.dtype)
    out = torch.matmul(weights, v)
    return out

def xformers_attention(q, k, v, attn_mask=None):
    B, H, L, D = q.shape
    q_ = q.reshape(B*H, L, D)
    k_ = k.reshape(B*H, L, D)
    v_ = v.reshape(B*H, L, D)

    bool_mask = None
    if attn_mask is not None:
        expanded = attn_mask.expand(B, H, L, L).reshape(B*H, L, L)
        bool_mask = (expanded < -1e4)
    out = xops.memory_efficient_attention(
        q_, k_, v_,
        attn_mask=bool_mask,
        p=0.0
    )
    return out.reshape(B, H, L, D)

def flash_attention(q, k, v, attn_mask=None):
    import flash_attn
    B, H, L, D = q.shape
    q_ = q.reshape(B*H, L, D)
    k_ = k.reshape(B*H, L, D)
    v_ = v.reshape(B*H, L, D)
    out = flash_attn.flash_attn_func(
        q_, k_, v_,
        dropout_p=0.0,
        softmax_scale=None,
        causal=False
    )
    return out.reshape(B, H, L, D)

def sdpa_attention(q, k, v, attn_mask=None):
    from torch.nn.functional import scaled_dot_product_attention as sdpa
    B, H, L, D = q.shape
    q_ = q.permute(2, 0, 1, 3).reshape(L, B*H, D)
    k_ = k.permute(2, 0, 1, 3).reshape(L, B*H, D)
    v_ = v.permute(2, 0, 1, 3).reshape(L, B*H, D)

    am = None
    if attn_mask is not None:
        am = attn_mask.expand(B, H, L, L).reshape(B*H, L, L)
    out_ = sdpa(q_, k_, v_, attn_mask=am, dropout_p=0.0, is_causal=False)
    out = out_.reshape(L, B, H, D).permute(1, 2, 0, 3)
    return out

def unified_attention(q, k, v, attn_mask=None, backend="auto"):
    if backend == "auto":
        if XFORMERS_AVAILABLE:
            backend = "xformers"
        elif FLASH_ATTN_AVAILABLE:
            backend = "flash"
        elif SDPA_AVAILABLE:
            backend = "sdpa"
        else:
            backend = "flex"

    if backend == "xformers":
        if not XFORMERS_AVAILABLE:
            raise RuntimeError("xformers not installed!")
        return xformers_attention(q, k, v, attn_mask)
    elif backend == "flash":
        if not FLASH_ATTN_AVAILABLE:
            raise RuntimeError("flash_attn not installed!")
        return flash_attention(q, k, v, attn_mask)
    elif backend == "sdpa":
        if not SDPA_AVAILABLE:
            raise RuntimeError("PyTorch >=2.0 needed for SDPA!")
        return sdpa_attention(q, k, v, attn_mask)
    else:
        return flex_custom_attention(q, k, v, attn_mask)

# Demo usage
def example_unified_attention():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    B, H, L, D = 2, 4, 16, 64
    q = torch.randn(B, H, L, D, device=device, dtype=torch.float16)
    k = torch.randn(B, H, L, D, device=device, dtype=torch.float16)
    v = torch.randn(B, H, L, D, device=device, dtype=torch.float16)
    attn_mask = torch.zeros((B, 1, L, L), device=device, dtype=torch.float32)
    blocked = torch.rand((B, 1, L, L), device=device) < 0.2
    attn_mask[blocked] = float("-inf")
    out_flex = unified_attention(q, k, v, attn_mask, backend="flex")
    print("fallback =>", out_flex.shape)

if __name__ == "__main__":
    example_unified_attention()


fallback => torch.Size([2, 4, 16, 64])


---
## Final Notes

- This notebookincludes separate code snippets for each task.
- Some cells (like the nF4 → Triton example) are skeletons or placeholders to illustrate core ideas.
